# 1. Cel i zakres projektu

### Cel

Głównym celem projektu jest analiza jakości powietrza w Polsce na podstawie danych dotyczących różnych wskaźników czystości powietrza (pyły PM2.5, PM10) zbieranych z różnych stacji pomiarowych na terenie kraju.

**Projekt ma na celu:**

    1. Identyfikację obszarów o najwyższym i najniższym poziomie zanieczyszczeń.

    2. Analizę trendów czasowych (np. sezonowych) w stężeniach zanieczyszczeń.

    3. Dostarczenie informacji dla decydentów i społeczeństwa, które mogą przyczynić się do poprawy jakości powietrza.

### Zakres

#### Dane: 
    Wykorzystane zostaną dane z różnych stacji pomiarowych na terenie Polski, zawierające informacje o stężeniach zanieczyszczeń powietrza (z powodu małej ilości stacji pomiarowych zbierające dane o innych zanieczyszczeniach zdecydowaliśmy się na analizy stężeń pyłów PM2.5 i PM10). Dane zostały pozyskane ze strony internetowej Głównego Inspektoratu Ochrony Środowiska (https://powietrze.gios.gov.pl).

#### Analiza: 
    Dane zostaną poddane analizie statystycznej i przestrzennej, w tym:
    
    - Mapowanie stężeń zanieczyszczeń.
    
    - Identyfikacja obszarów o najwyższym i najniższym poziomie zanieczyszczeń.
    
    - Analiza trendów czasowych.

#### Wyniki: 
    Projekt dostarczy wizualizacje, które będą mogły być wykorzystane do podejmowania decyzji dotyczących ochrony środowiska i zdrowia publicznego.

## 2. Dataset

### Opis danych
Każdy rekord z rekordów jest uśrednionym pomiarem danego wskaźnika z 24 godzin, zrealizowanym w konkretnej lokalizacji.

| Nazwa kolumny       | Opis                                                                 |
|---------------------|---------------------------------------------------------------------|
| `kod_stacji`        | Unikalny kod identyfikujący stację pomiarową.                       |
| `wskaznik`          | Rodzaj mierzonego wskaźnika (np. PM2.5, PM10, NO2).                 |
| `czas_usredniania`  | Okres uśredniania pomiarów (np. 1 godzina, 24 godziny).             |
| `jednostka`         | Jednostka miary dla wskaźnika (np. µg/m³).                          |
| `data`              | Data pomiaru.                                                       |
| `pomiar`            | Zmierzona wartość wskaźnika.                                        |
| `stary_kod_stacji`  | Poprzedni kod stacji (jeśli istnieje).                              |
| `typ_obszaru`       | Typ obszaru, na którym znajduje się stacja (np. miejski, wiejski).  |
| `rodzaj_stacji`     | Rodzaj stacji pomiarowej (np. tło, komunikacyjna).                  |
| `wojewodztwo`       | Województwo, w którym znajduje się stacja.                          |
| `miejscowosc`       | Miejscowość, w której znajduje się stacja.                          |
| `latitude`          | Szerokość geograficzna stacji pomiarowej.                           |
| `longitude`         | Długość geograficzna stacji pomiarowej.                             |

Dataset został stworzony wstępnie w programie Excel (transpozycje, usunięcie zbędnych kolumn), a następnie przekształcony do potrzebnego nam formatu w pythonie za pomocą biblioteki pandas:
```python
import pandas as pd

def merge_measures_with_stations_data(df_data, df_stations_data, new_file_name: str, year: str):
    df_data_melted = df_data.melt(id_vars=['kod_stacji', 'wskaznik', 'czas_usredniania', 'jednostka'], var_name='data', value_name='pomiar')
    merged = pd.merge(df_data_melted, df_stations_data, on='kod_stacji', how='inner')
    merged['data'] = pd.to_datetime(merged['data'], format='%d-%m-%Y')
    merged.to_csv(f'data/{year}/{new_file_name}.csv', index=False)

df_stations = pd.read_excel('data/metadane_kody_stacji_i_stanowisk_pomiarowych.xlsx')
df_2023_pm_10 = pd.read_excel('data/2023/2023_PM10_24h_formatted.xlsx')
df_2022_pm_10 = pd.read_excel('data/2022/2022_PM10_24h_formatted.xlsx')
df_2021_pm_10 = pd.read_excel('data/2021/2021_PM10_24h_formatted.xlsx')

df_2023_pm_25 = pd.read_excel('data/2023/2023_PM25_24h_formatted.xlsx')
df_2022_pm_25 = pd.read_excel('data/2022/2022_PM25_24h_formatted.xlsx')
df_2021_pm_25 = pd.read_excel('data/2021/2021_PM25_24h_formatted.xlsx')

merge_measures_with_stations_data(df_2023_pm_10, df_stations, '2023_pm10_merged', '2023')
merge_measures_with_stations_data(df_2022_pm_10, df_stations, '2022_pm10_merged', '2022')
merge_measures_with_stations_data(df_2021_pm_10, df_stations, '2021_pm10_merged', '2021')

merge_measures_with_stations_data(df_2023_pm_10, df_stations, '2023_pm25_merged', '2023')
merge_measures_with_stations_data(df_2022_pm_10, df_stations, '2022_pm25_merged', '2022')
merge_measures_with_stations_data(df_2021_pm_10, df_stations, '2021_pm25_merged', '2021')
```

Dane geometryczne są w systemie odniesienia: **WGS84**, oparty na nim system przestrzenny to: **EPSG:4326**.

# 2. Analiza i wizualizacja

### Wczytanie danych

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, MarkerCluster, TimestampedGeoJson
import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sns
import mapclassify
import calendar
import warnings

sns.set_style("darkgrid")
warnings.filterwarnings('ignore')

In [ ]:
df_2023_pm_10 = pd.read_csv('data/2023/2023_pm10_merged.csv') 
df_2022_pm_10 = pd.read_csv('data/2022/2022_pm10_merged.csv') 
df_2021_pm_10 = pd.read_csv('data/2021/2021_pm10_merged.csv')

df_2023_pm_25 = pd.read_csv('data/2023/2023_pm25_merged.csv') 
df_2022_pm_25 = pd.read_csv('data/2022/2022_pm25_merged.csv') 
df_2021_pm_25 = pd.read_csv('data/2021/2021_pm25_merged.csv')

In [ ]:
geometry_pm_10_2023 = gpd.points_from_xy(df_2023_pm_10['longitude'], df_2023_pm_10['latitude'])
geometry_pm_10_2022 = gpd.points_from_xy(df_2022_pm_10['longitude'], df_2022_pm_10['latitude'])
geometry_pm_10_2021 = gpd.points_from_xy(df_2021_pm_10['longitude'], df_2021_pm_10['latitude'])

geometry_pm_25_2023 = gpd.points_from_xy(df_2023_pm_25['longitude'], df_2023_pm_25['latitude'])
geometry_pm_25_2022 = gpd.points_from_xy(df_2022_pm_25['longitude'], df_2022_pm_25['latitude'])
geometry_pm_25_2021 = gpd.points_from_xy(df_2021_pm_25['longitude'], df_2021_pm_25['latitude'])

In [ ]:
gdf_2023_pm_10 = gpd.GeoDataFrame(df_2023_pm_10, geometry=geometry_pm_10_2023)
gdf_2022_pm_10 = gpd.GeoDataFrame(df_2022_pm_10, geometry=geometry_pm_10_2022)
gdf_2021_pm_10 = gpd.GeoDataFrame(df_2021_pm_10, geometry=geometry_pm_10_2021)

gdf_2023_pm_25 = gpd.GeoDataFrame(df_2023_pm_25, geometry=geometry_pm_25_2023)
gdf_2022_pm_25 = gpd.GeoDataFrame(df_2022_pm_25, geometry=geometry_pm_25_2022)
gdf_2021_pm_25 = gpd.GeoDataFrame(df_2021_pm_25, geometry=geometry_pm_25_2021)

In [ ]:
gdf_2023_pm_10.set_crs(epsg=4326, inplace=True)
gdf_2022_pm_10.set_crs(epsg=4326, inplace=True)
gdf_2021_pm_10.set_crs(epsg=4326, inplace=True);

gdf_2023_pm_25.set_crs(epsg=4326, inplace=True)
gdf_2022_pm_25.set_crs(epsg=4326, inplace=True)
gdf_2021_pm_25.set_crs(epsg=4326, inplace=True);

### Dostępność punktów pomiarowych względem lat

In [ ]:
measuring_stations_2023_num = len(gdf_2023_pm_10.kod_stacji.unique())
measuring_stations_2022_num = len(gdf_2022_pm_10.kod_stacji.unique())
measuring_stations_2021_num = len(gdf_2021_pm_10.kod_stacji.unique())
print(f'Liczba stacji pomiarowych w roku 2023: {measuring_stations_2023_num}.')
print(f'Liczba stacji pomiarowych w roku 2022: {measuring_stations_2022_num}.')
print(f'Liczba stacji pomiarowych w roku 2021: {measuring_stations_2021_num}.')

**Ze wstępnej analizy wynika, że liczba stacji pomiarowych (pył PM10) maleje z roku na rok.**

### Wizualizacja dostępnych stacji w poszczególnych latach

In [ ]:
gdf_2021_web_mercator = gdf_2021_pm_10.to_crs(epsg=3857)
gdf_2022_web_mercator = gdf_2022_pm_10.to_crs(epsg=3857)
gdf_2023_web_mercator = gdf_2023_pm_10.to_crs(epsg=3857)

fig, axes = plt.subplots(3, 1, figsize=(20, 15))
years = [("2021", gdf_2021_web_mercator), ("2022", gdf_2022_web_mercator), ("2023", gdf_2023_web_mercator)]

for ax, (year, gdf) in zip(axes, years):
    gdf.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.7)
    
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    ax.set_title(f"Dane dla roku {year}", fontsize=14)

plt.tight_layout()
plt.show()

### Przygotowanie danych do map interaktywnych

In [ ]:
coordinates_2021 = gdf_2021_pm_10[['latitude', 'longitude']].values.tolist()
coordinates_2022 = gdf_2022_pm_10[['latitude', 'longitude']].values.tolist()
coordinates_2023 = gdf_2023_pm_10[['latitude', 'longitude']].values.tolist()

coordinates_2021 = list(set(map(tuple, coordinates_2021)))
coordinates_2022 = list(set(map(tuple, coordinates_2022)))
coordinates_2023 = list(set(map(tuple, coordinates_2023)))

poland_center_lat = 51.9194
poland_center_lon = 19.1451

### Mapy ciepła dla stacji pomiarowych w poszczególnych latach

In [ ]:
m = folium.Map(location=[poland_center_lat, poland_center_lon], zoom_start=6, control_scale=True)

fg_2021 = folium.FeatureGroup(name="Heatmap 2021", show=False)
fg_2022 = folium.FeatureGroup(name="Heatmap 2022", show=False)
fg_2023 = folium.FeatureGroup(name="Heatmap 2023", show=False)

HeatMap(coordinates_2021, radius=15).add_to(fg_2021)
HeatMap(coordinates_2022, radius=15).add_to(fg_2022)
HeatMap(coordinates_2023, radius=15).add_to(fg_2023)

fg_2021.add_to(m)
fg_2022.add_to(m)
fg_2023.add_to(m)

folium.LayerControl().add_to(m)

m

### Interaktywna wizualizacja dostępnych stacji w poszczególnych latach (zgrupowane znaczniki)

In [ ]:
m = folium.Map(location=[poland_center_lat, poland_center_lon], zoom_start=6, control_scale=True)

fg_2021 = folium.FeatureGroup(name="2021", show=False)
fg_2022 = folium.FeatureGroup(name="2022", show=False)
fg_2023 = folium.FeatureGroup(name="2023", show=False)

marker_cluster_2021 = MarkerCluster(name="Stacje pomiarowe 2021").add_to(fg_2021)
marker_cluster_2022 = MarkerCluster(name="Stacje pomiarowe 2022").add_to(fg_2022)
marker_cluster_2023 = MarkerCluster(name="Stacje pomiarowe 2023").add_to(fg_2023)

unique_stations_2021 = gdf_2021_pm_10.groupby('kod_stacji').first().reset_index()
unique_stations_2022 = gdf_2022_pm_10.groupby('kod_stacji').first().reset_index()
unique_stations_2023 = gdf_2023_pm_10.groupby('kod_stacji').first().reset_index()

def get_icon(row):
    if row['typ_obszaru'] == 'miejski':
        return folium.Icon(color='blue', icon='city', prefix='fa')
    elif row['typ_obszaru'] == 'podmiejski':
        return folium.Icon(color='green', icon='tree', prefix='fa')
    elif row['typ_obszaru'] == 'pozamiejski':
        return folium.Icon(color='lightblue', icon='tree-city', prefix='fa')

for _, row in unique_stations_2021.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['miejscowosc'],
        popup=f"Kod stacji: {row['kod_stacji']}<br><br>Typ obszaru: {row['typ_obszaru']}<br><br>Rodzaj stacji: {row['rodzaj_stacji']}",
        icon=get_icon(row)
    ).add_to(marker_cluster_2021)

for _, row in unique_stations_2022.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['miejscowosc'],
        popup=f"Kod stacji: {row['kod_stacji']}<br><br>Typ obszaru: {row['typ_obszaru']}<br><br>Rodzaj stacji: {row['rodzaj_stacji']}",
        icon=get_icon(row)
    ).add_to(marker_cluster_2022)

for _, row in unique_stations_2023.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['miejscowosc'],
        popup=f"Kod stacji: {row['kod_stacji']}<br><br>Typ obszaru: {row['typ_obszaru']}<br><br>Rodzaj stacji: {row['rodzaj_stacji']}",
        icon=get_icon(row)
    ).add_to(marker_cluster_2023)

fg_2021.add_to(m)
fg_2022.add_to(m)
fg_2023.add_to(m)

folium.LayerControl().add_to(m)

m

##### Po wstępnej analizie (wizualnej) powyższych map postanowiliśmy sprawdzić jak zmienia się liczba aktywnych stacji pomiarowych w poszczególnych województwach.

In [ ]:
measuring_stations_2023_by_voivodeships = df_2023_pm_10.groupby('wojewodztwo')['kod_stacji'].nunique()
measuring_stations_2022_by_voivodeships = df_2022_pm_10.groupby('wojewodztwo')['kod_stacji'].nunique()
measuring_stations_2021_by_voivodeships = df_2021_pm_10.groupby('wojewodztwo')['kod_stacji'].nunique()
print(f'Rok 2023:\n{measuring_stations_2023_by_voivodeships}')
print('='*30)
print(f'Rok 2022:\n{measuring_stations_2022_by_voivodeships}')
print('='*30)
print(f'Rok 2021:\n{measuring_stations_2021_by_voivodeships}')

In [ ]:
measuring_stations_by_voivodeships = pd.DataFrame({
    '2021': measuring_stations_2021_by_voivodeships,
    '2022': measuring_stations_2022_by_voivodeships,
    '2023': measuring_stations_2023_by_voivodeships
}).fillna(0).astype(int)

measuring_stations_by_voivodeships['Difference_2022_2021'] = measuring_stations_by_voivodeships['2022'] - measuring_stations_by_voivodeships['2021']
measuring_stations_by_voivodeships['Difference_2023_2022'] = measuring_stations_by_voivodeships['2023'] - measuring_stations_by_voivodeships['2022']
measuring_stations_by_voivodeships['Difference_2023_2021'] = measuring_stations_by_voivodeships['2023'] - measuring_stations_by_voivodeships['2021']

measuring_stations_by_voivodeships

In [ ]:
measuring_stations_by_voivodeships.describe()

Na podstawie powyższych tabel można zauważyć, że każde województwo posiada co najmniej cztery stacje pomiarowe (województwa opolskie i polaskie). Najwięcej stacji pomiarowych znajduje się natomiast w województwie małopolskim. W zakresie zmienności liczby stacji pomiarowych, odnotowano maksymalny wzrost o jedną stację, a maksymalny spadek na poziomie dwóch stacji.

### Analiza jakości powietrza

**Ocena jakości powietrza w stosunku do stężenia pyłu PM10 i PM2.5 (dane z GIOŚ):**

**PM10**
| Zakres stężeń (µg/m³) | Kategoria jakości powietrza |
|------------------------|-----------------------------|
| 0 - 20                 | Bardzo dobry                |
| 20.1 - 50              | Dobry                       |
| 50.1 - 80              | Umiarkowany                 |
| 80.1 - 110             | Dostateczny                 |
| 110.1 - 150            | Zły                         |
| > 150                  | Bardzo zły                  |

**PM2.5**
| Zakres stężenia (µg/m³) | Poziom jakości powietrza |
|-------------------------|-------------------------|
| 0 - 13                 | Bardzo dobry           |
| 13.1 - 35              | Dobry                  |
| 35.1 - 55              | Umiarkowany            |
| 55.1 - 75              | Dostateczny            |
| 75.1 - 110             | Zły                    |
| > 110                  | Bardzo zły             |


In [ ]:
gdf_2023_pm_10_cleaned = gdf_2023_pm_10.dropna(subset=['pomiar'])
gdf_2022_pm_10_cleaned = gdf_2022_pm_10.dropna(subset=['pomiar'])
gdf_2021_pm_10_cleaned = gdf_2021_pm_10.dropna(subset=['pomiar'])

gdf_2023_pm_25_cleaned = gdf_2023_pm_25.dropna(subset=['pomiar'])
gdf_2022_pm_25_cleaned = gdf_2022_pm_25.dropna(subset=['pomiar'])
gdf_2021_pm_25_cleaned = gdf_2021_pm_25.dropna(subset=['pomiar'])

bins = [0, 20, 50, 80, 110, 150]
classifier = mapclassify.UserDefined.make(bins=bins)

categories = {
    1: 'Bardzo dobry',
    2: 'Dobry',
    3: 'Umiarkowany',
    4: 'Dostateczny',
    5: 'Zły',
    6: 'Bardzo zły'
}

gdf_2023_pm_10_cleaned['class'] = classifier(gdf_2023_pm_10_cleaned['pomiar'])
gdf_2022_pm_10_cleaned['class'] = classifier(gdf_2022_pm_10_cleaned['pomiar'])
gdf_2021_pm_10_cleaned['class'] = classifier(gdf_2021_pm_10_cleaned['pomiar'])


gdf_2023_pm_10_cleaned['jakosc_powietrza'] = gdf_2023_pm_10_cleaned['class'].map(categories)
gdf_2022_pm_10_cleaned['jakosc_powietrza'] = gdf_2022_pm_10_cleaned['class'].map(categories)
gdf_2021_pm_10_cleaned['jakosc_powietrza'] = gdf_2021_pm_10_cleaned['class'].map(categories)

In [ ]:
gdf_voivodeships = gpd.read_file('data/wojewodztwa/wojewodztwa.shp')
gdf_voivodeships = gdf_voivodeships.to_crs(epsg=4326)

In [ ]:
NUM_OF_VOIVODESHIPS = 16
assert len(gdf_voivodeships['JPT_NAZWA_'].unique()) == NUM_OF_VOIVODESHIPS, 'Not all voivodeships available'
assert len(gdf_2023_pm_10['wojewodztwo'].unique()) == NUM_OF_VOIVODESHIPS, 'Not all voivodeships available'
assert len(gdf_2022_pm_10['wojewodztwo'].unique()) == NUM_OF_VOIVODESHIPS, 'Not all voivodeships available'
assert len(gdf_2021_pm_10['wojewodztwo'].unique()) == NUM_OF_VOIVODESHIPS, 'Not all voivodeships available'

In [ ]:
gdf_voivodeships['wojewodztwo'] = gdf_voivodeships['JPT_NAZWA_'].str.upper()

#### Średnie roczne stężenie pyłu PM10 i PM2.5 w poszczególnych województwach

In [ ]:
mean_year_air_quality_pm_10_2023 = gdf_2023_pm_10_cleaned.groupby('wojewodztwo')['pomiar'].mean()
mean_year_air_quality_pm_10_2022 = gdf_2022_pm_10_cleaned.groupby('wojewodztwo')['pomiar'].mean()
mean_year_air_quality_pm_10_2021 = gdf_2021_pm_10_cleaned.groupby('wojewodztwo')['pomiar'].mean()

gdf_voivodeships['mean_year_pm10_2023'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_10_2023)
gdf_voivodeships['mean_year_pm10_2022'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_10_2022)
gdf_voivodeships['mean_year_pm10_2021'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_10_2021)

mean_year_air_quality_pm_25_2023 = gdf_2023_pm_25_cleaned.groupby('wojewodztwo')['pomiar'].mean()
mean_year_air_quality_pm_25_2022 = gdf_2022_pm_25_cleaned.groupby('wojewodztwo')['pomiar'].mean()
mean_year_air_quality_pm_25_2021 = gdf_2021_pm_25_cleaned.groupby('wojewodztwo')['pomiar'].mean()

gdf_voivodeships['mean_year_pm25_2023'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_25_2023)
gdf_voivodeships['mean_year_pm25_2022'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_25_2022)
gdf_voivodeships['mean_year_pm25_2021'] = gdf_voivodeships['wojewodztwo'].map(mean_year_air_quality_pm_25_2021)

bins_pm_10 = [0, 20, 50, 80, 110, 150]
bins_pm_25 = [0, 13, 35, 55, 75, 110]

m = folium.Map(location=[poland_center_lat, poland_center_lon], zoom_start=6, control_scale=True)

chloropleth_pm_10_2023 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM10 w roku 2023',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm10_2023'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Średnie roczne stężenie PM10 w powietrzu',
    bins=bins_pm_10,
    show=True
).add_to(m)

chloropleth_pm_10_2022 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM10 w roku 2022',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm10_2022'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins_pm_10,
    show=False
)

chloropleth_pm_10_2021 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM10 w roku 2021',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm10_2021'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins_pm_10,
    show=False
).add_to(m)

# remove legends
for key in chloropleth_pm_10_2022._children:
    if key.startswith('color_map'):
        del(chloropleth_pm_10_2022._children[key])
chloropleth_pm_10_2022.add_to(m)     

for key in chloropleth_pm_10_2021._children:
    if key.startswith('color_map'):
        del(chloropleth_pm_10_2021._children[key])
chloropleth_pm_10_2021.add_to(m)  

chloropleth_pm_25_2023 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM2.5 w roku 2023',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm25_2023'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Średnie roczne stężenie PM2.5 w powietrzu',
    bins=bins_pm_25,
    show=False
).add_to(m)

chloropleth_pm_25_2022 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM2.5 w roku 2022',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm25_2022'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins_pm_25,
    show=False
)

chloropleth_pm_25_2021 = folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Średnie stężenie PM2.5 w roku 2021',
    data=gdf_voivodeships,
    columns=['wojewodztwo', 'mean_year_pm25_2021'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins_pm_25,
    show=False
).add_to(m)

# remove legends
for key in chloropleth_pm_25_2022._children:
    if key.startswith('color_map'):
        del(chloropleth_pm_25_2022._children[key])
chloropleth_pm_25_2022.add_to(m)     

for key in chloropleth_pm_25_2021._children:
    if key.startswith('color_map'):
        del(chloropleth_pm_25_2021._children[key])
chloropleth_pm_25_2021.add_to(m)  

folium.LayerControl().add_to(m)

m

Na powyższej mapie możemy zaobserwować że uśrednione roczne stężeniu pyłu PM10 w latach 2021 i 2022 w większości województw jest klasyfikowana jako **Dobra** natomiast w województwie zachodniopomorskim jako **Bardzo dobra**. W roku 2023 możemy zaobserwować aż 8 województw w których jakość powietrze jest **Bardzo dobra**, a w pozostałych **Dobra**.

#### Analiza liczby przypadających stacji na powierzchnię (w obrębie województwa)

In [ ]:
gdf_voivodeships = gdf_voivodeships.to_crs(epsg=2180)
gdf_voivodeships['powierzchnia_km2'] = gdf_voivodeships.geometry.area / 1_000_000
gdf_voivodeships = gdf_voivodeships.to_crs(epsg=4326)

In [ ]:
df_voivodeships_popn = pd.read_excel('data/populacja/populacja_w_wojewodztwach.xlsx')

stations_per_voivodeship = gdf_2023_pm_10_cleaned.groupby('wojewodztwo')['kod_stacji'].nunique().reset_index()
stations_per_voivodeship.rename(columns={'kod_stacji': 'liczba_stacji'}, inplace=True)

gdf_2023_pm_10_area = pd.merge(stations_per_voivodeship, df_voivodeships_popn, how='inner', left_on='wojewodztwo', right_on='voivodeship')

gdf_2023_pm_10_area = pd.merge(gdf_voivodeships[['wojewodztwo', 'powierzchnia_km2']], gdf_2023_pm_10_area, how='inner', left_on='wojewodztwo', right_on='voivodeship')
gdf_2023_pm_10_area.drop(columns=['wojewodztwo_x', 'wojewodztwo_y'], axis=1, inplace=True)

gdf_2023_pm_10_area = pd.merge(gdf_voivodeships[['wojewodztwo', 'mean_year_pm10_2023']], gdf_2023_pm_10_area, how='inner', left_on='wojewodztwo', right_on='voivodeship')

gdf_2023_pm_10_area['stations_per_1000km2'] = gdf_2023_pm_10_area['liczba_stacji'] / gdf_2023_pm_10_area['powierzchnia_km2'] * 1_000
gdf_2023_pm_10_area = gdf_2023_pm_10_area[['voivodeship', 'liczba_stacji','powierzchnia_km2', 'population',
                                           'stations_per_1000km2', 'mean_year_pm10_2023']]

gdf_2023_pm_10_area[['population', 'mean_year_pm10_2023']].corr()

W powyższym podpunkcie została obliczona liczba stacji przypadających na 1000km2 (w obrębie województwa). Dodatkowo sprawdziliśmy czy występuje korelacja pomiędzy liczbą ludności w województwie a średnim rocznym stężeniem pyłu PM10 w roku 2023 (dla województwa). Obliczona korelacja (pearsona) jest korelacją silną (>0.5), możemy zatem wysunąć wniosek, że liczba ludności ma wpływ na zanieczyszczenie powietrza. Niestety dane, które posiadamy nie pozwalają na dalszą analizę w tym kierunku.

In [ ]:
sns.barplot(data=gdf_2023_pm_10_area, y='voivodeship', x='stations_per_1000km2')
plt.xlabel('Liczba stacji na 1000 km2')
plt.ylabel('Województwa')
plt.title('Liczba stacji pomiaru stężenia pyłu PM10 (na 1000 km2)')

plt.show()

In [ ]:
m = folium.Map(location=[poland_center_lat, poland_center_lon], zoom_start=6, control_scale=True)

folium.Choropleth(
    geo_data=gdf_voivodeships,
    name='Stacje na 1000 km²',
    data=gdf_2023_pm_10_area,
    columns=['voivodeship', 'stations_per_1000km2'],
    key_on='feature.properties.wojewodztwo',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Liczba stacji na 1000 km2',
    show=True
).add_to(m)

folium.LayerControl().add_to(m)

m

W związku z obliczonym współczynnikiem liczby stacji przypadających na 1000km2 stworzyliśmy mapę pokazującą zróżnicowanie jego wartości względem województw. Wnioskując z powyższej mapy największa gęstość stacji pomiarowych występuje w województwach małopolskim i śląskim. Prawdopodobnie jest to związane z dominującymi sektorami gospodarki i charakterystyką ukształtowania terenu. Na podstawie powyższej mapy podejrzewamy, że instnieją w Polsce obszary które w żadnym stopniu nie są monitorowane (opierając analizę na dostępnych dla nas danych - informacje na stronie GIOŚ sugerują, że stacji pomiarowych jest znacznie więcej, nie znajdują się one jednak w analizowanym zbiorze danych).

Opierając się na naszym podejrzeniu dotyczącym brakiem stacji na niektórych terenach stworzyliśmy mapę zaznaczającą promień efektywnych pomiarów stężenia pyłu PM10 w powietrzu, niestety GIOŚ nie udostępnia takich informacji. Przyjeliśmy więc odpowiednio dla obszarów miejskich: 1km, dla podmiejskich: 2.5km, pozamiejskich: 10km. Takie wartości nie pozwalają na pokrycie całego obszaru Polski, co więcej nawet wartości podniesione o jeden rząd wielkości nie spełniają tego warunku. Jest zatem wysoko prawdopodobne, że niektóre obszary Polski nie są monitorowane.

In [ ]:
gdf_2023_pm_10_unique = gdf_2023_pm_10.drop_duplicates(subset='kod_stacji')

buffer_settings = {
    'miejski': {'radius': 1000, 'color': 'red'},
    'podmiejski': {'radius': 2500, 'color': 'orange'},
    'pozamiejski': {'radius': 10000, 'color': 'green'}
}

m = folium.Map(location=[poland_center_lat, poland_center_lon], zoom_start=6, control_scale=True)

buffer_groups = {
    typ: folium.FeatureGroup(name=f'Promień {typ} ({radius["radius"]/1000} km)', show=False)
    for typ, radius in buffer_settings.items()
}
def get_icon(row):
    if row['typ_obszaru'] == 'miejski':
        return folium.Icon(color='blue', icon='city', prefix='fa')
    elif row['typ_obszaru'] == 'podmiejski':
        return folium.Icon(color='green', icon='tree', prefix='fa')
    elif row['typ_obszaru'] == 'pozamiejski':
        return folium.Icon(color='lightblue', icon='tree-city', prefix='fa')


for _, row in gdf_2023_pm_10_unique.iterrows():
    
    lat, lon = row.geometry.y, row.geometry.x
    
    popup = folium.Popup(
        f"<b>{row['miejscowosc']}</b><br>"
        f"Typ obszaru: {row['typ_obszaru']}<br>"
        f"Kod stacji: {row['kod_stacji']}",
        max_width=250
    )
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        icon=get_icon(row)
    ).add_to(m)
    
    typ = row['typ_obszaru']
    if typ in buffer_settings:
        folium.Circle(
            location=[lat, lon],
            radius=buffer_settings[typ]['radius'],
            color=buffer_settings[typ]['color'],
            fill=True,
            fill_opacity=0.2,
            tooltip=f"Promień {typ}: {buffer_settings[typ]['radius']} m"
        ).add_to(buffer_groups[typ])

for group in buffer_groups.values():
    group.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

#### Klasyfikacja jakości powietrza na podstawie stężenia pyłu PM10

Poniższa animacja przedstawia dzienne zmiany wskaźnika jakości powietrza na podstawie stężenia pyłu PM10 na przestrzeni ostatnich trzech lat.

Jakość powietrza dla tej mapy jest klasyfikowana na podstawie stężenia pyłu PM10 według następujących przedziałów:

| Stężenie PM10 (µg/m³) | Klasyfikacja jakości powietrza | Kolor oznaczenia |
|----------------------|--------------------------------|------------------------|
| **0 – 20**           | **Bardzo dobra**               | 🟢 Jasnozielony       |
| **20.1 – 50**        | **Dobra**                      | 🟡 Żółty              |
| **50.1 – 80**        | **Umiarkowana**                | 🟠 Pomarańczowy       |
| **80.1 – 110**       | **Dostateczna**                | 🟤 Ciemnopomarańczowy |
| **110.1 – 150**      | **Zła**                        | 🔴 Czerwony           |
| **> 150**            | **Bardzo zła**                 | 🔴 Ciemnoczerwony     |

Każdy poziom jakości powietrza jest oznaczony odpowiednim kolorem, co ułatwia wizualizację stopnia zanieczyszczenia.

In [ ]:
df_2023_pm_10 = pd.read_csv('data/2023/2023_pm10_merged.csv')
df_2022_pm_10 = pd.read_csv('data/2022/2022_pm10_merged.csv')
df_2021_pm_10 = pd.read_csv('data/2021/2021_pm10_merged.csv')

df_combined = pd.concat([df_2023_pm_10, df_2022_pm_10, df_2021_pm_10])

df_combined['data'] = pd.to_datetime(df_combined['data'])

gdf_combined = gpd.GeoDataFrame(df_combined, geometry=gpd.points_from_xy(df_combined.longitude, df_combined.latitude))
gdf_combined.set_crs(epsg=4326, inplace=True)

bins = [0, 20, 50, 80, 110, 150]
colors = ['#66FF66', '#FFFF00', '#FFCC00', '#FF9900', '#FF0000', '#990000']
classifier = mapclassify.UserDefined.make(bins=bins)

gdf_combined['class'] = classifier(gdf_combined['pomiar'])

bin_colors = {i: color for i, color in enumerate(colors, 1)}

m = folium.Map(location=[51.9194, 19.1451], zoom_start=6, control_scale=True)

features = []
for _, row in gdf_combined.iterrows():
    class_value = row['class']
    if class_value in bin_colors:
        color = bin_colors[class_value]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [row['longitude'], row['latitude']],
            },
            'properties': {
                'time': row['data'].isoformat(),
                'popup': f"Kod stacji: {row['kod_stacji']}<br>Pomiar: {row['pomiar']} µg/m³",
                'icon': 'circle',
                'iconstyle': {
                    'fillColor': color,
                    'fillOpacity': 1,
                    'stroke': 'true',
                    'radius': 7
                }
            }
        }
        features.append(feature)

TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='P1D', add_last_point=True, auto_play=True, loop=True).add_to(m)

m

Obserwując animację można zauważyć, że w miesiącach zimowych i jesiennych jakość powietrze znacznie się pogarsza, przedstawiamy ten trend na poniższym wykresie.

In [ ]:
df_combined['month'] = df_combined['data'].dt.month

pp = df_combined.groupby('month')['pomiar'].mean().reset_index()

plt.figure(figsize=(12, 6))

sns.lineplot(data=pp, x='month', y='pomiar')

plt.xlabel('Miesiąc')
plt.ylabel('Średni pomiar')
plt.title('Średni pomiar stężenia pyłu PM10 w poszczególnych miesiącach')

plt.xticks(ticks=range(1, 13), labels=[calendar.month_name[i] for i in range(1, 13)], rotation=45)

plt.show()

# 3. Wnioski

Na podstawie powyższych analiz możemy stwierdzić że:
- prawdopodobnie ilość stacji pomiaru zanieczyszczeń powietrza w Polsce jest niewystarczająca
- liczba ludności na danych obszarach ma silny wpływ na zanieczyszczenie powietrza (temat do dalszej analizy)
- w miesiącach zimowych i jesiennych poziom zanieczyszczenia powietrze przerażająco rośnie (temat do dalszej analizy)